In [2]:
import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None   # default='warn'
data = pd.read_excel('cleaned_data.xlsx')

In [3]:
data['Price_per_unit_area'] = data['Price_per_unit_area'].astype(str).str.replace(',', '').astype(int)
data['Price'] = data['Price'].astype(str).str.replace(',', '').astype(int)
data['Size'].astype(str)
data['Size'] = data['Size'].str.removesuffix(' sq ft').str.replace(',', '').astype(int)

In [4]:
data2 = data.copy()

In [53]:
data = data2.copy()

In [54]:
attrbs = ['Property_Name', 'Property_id', 'Project_URL', 'builder_id', 'Builder_name', 'Locality_ID', 'Longitude', 'Latitude', 'Sub_urban_ID', 'description', 'City_id', 'Listing_Category', 'is_plot', 'is_RERA_registered', 'is_Apartment', 'is_ready_to_move', 'is_commercial_Listing', 'is_PentaHouse', 'is_studio', 'listing_domain_score', 'No_of_BHK', 'Locality_Name', 'Sub_urban_name', 'Posted_On', 'Price_per_unit_area', 'Property_building_status', 'is_furnished']

for att in attrbs:
    data.drop(att, axis=1, inplace=True)

In [55]:
underConstruction = data[data['Property_status'] == 'Under Construction']
readyToMove = data[data['Property_status'] == 'Ready to move']

In [56]:
def priceG(dataset):
    _gold = ((data['Price'] <= 8000000000) & (data['Price'] >= 50000000))
    _silver = ((data['Price'] < 50000000) & (data['Price'] >= 2000000))
    _bronze = ((data['Price'] < 2000000) & (data['Price'] >= 100000))

    dataset.loc[_bronze, ['Price']] = 'Bronze'
    dataset.loc[_silver, ['Price']] = 'Silver'
    dataset.loc[_gold, ['Price']] = 'Gold'

In [57]:
def sizeG(dataset):
    _large = ((data['Size'] <= 90000) & (data['Size'] >= 50000))
    _medium = ((data['Size'] < 50000) & (data['Size'] >= 5000))
    _small = ((data['Size'] < 5000) & (data['Size'] >= 100))

    dataset.loc[_large, ['Size']] = 'Large'
    dataset.loc[_medium, ['Size']] = 'Medium'
    dataset.loc[_small, ['Size']] = 'Small'

In [58]:
def work(a, b):
    priceG(a)
    priceG(b)
    sizeG(a)
    sizeG(b)

In [59]:
work(underConstruction, readyToMove)
readyToMove = readyToMove.drop(['City_name'], axis=1)
underConstruction = underConstruction.drop(['City_name'], axis=1)

readyToMove = readyToMove.drop_duplicates()
underConstruction = underConstruction.drop_duplicates()


/tmp/ipykernel_71677/2737076903.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[_bronze, ['Price']] = 'Bronze'
/tmp/ipykernel_71677/2737076903.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[_bronze, ['Price']] = 'Bronze'
/tmp/ipykernel_71677/72417587.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [60]:
_underCost = []
_readyToMove = []

for i in range(len(underConstruction)):
    for j in range(len(readyToMove)):
        if underConstruction.iloc[i]['Price'] == readyToMove.iloc[j]['Price'] and underConstruction.iloc[i]['Property_type'] == readyToMove.iloc[j]['Property_type'] and underConstruction.iloc[i]['Size'] == readyToMove.iloc[j]['Size']:
            _underCost.append(i)

for i in range(len(readyToMove)):
    for j in range(len(underConstruction)):
        if readyToMove.iloc[i]['Price'] == underConstruction.iloc[j]['Price'] and readyToMove.iloc[i]['Property_type'] == underConstruction.iloc[j]['Property_type'] and readyToMove.iloc[i]['Size'] == underConstruction.iloc[j]['Size']:
            _readyToMove.append(i)

underConstruction.drop(underConstruction.index[_underCost], inplace=True)
readyToMove.drop(readyToMove.index[_readyToMove], inplace=True)

In [61]:
underConstruction

,Property_type,Property_status,Price,Size
475,Residential Plot,Under Construction,Silver,Large


We infer the rule *"Residential plots under construction are of silver price and large size"*. Converting it into a logic formula, we get:

$$\forall \; (x) \; \text{UnderConstruction}(x) <= \text{Property Type}(x) \in \text{Residential Plot} \wedge \text{Price}(x) \in \text{silver} \wedge \text{Size}(x) \in \text{Large}$$

In [62]:
readyToMove

,Property_type,Property_status,Price,Size
3758,Residential Plot,Ready to move,Gold,Small


We infer the rule *"Residential plots ready to move are of gold price and small size"*. Converting it into a logic formula, we get:

$$\forall \; (x) \; \text{Ready to Move}(x) <= \text{Property Type}(x) \in \text{Residential Plot} \wedge \text{Price}(x) \in \text{gold} \wedge \text{Size}(x) \in \text{small}$$

In [63]:
tech_city = data[(data['City_name'] == 'Hyderabad') | (data['City_name'] == 'Bangalore') | (data['City_name'] == 'Ahmedabad') | (data['City_name'] == 'Mumbai')]
tech_city['City_name'] = 'Tech City'
other_city = data[(data['City_name'] == 'Chennai') | (data['City_name'] == 'Kolkata') | (data['City_name'] == 'Delhi') | (data['City_name'] == 'Lucknow')]
other_city['City_name'] = 'Other City'

/tmp/ipykernel_71677/941310139.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tech_city['City_name'] = 'Tech City'
/tmp/ipykernel_71677/941310139.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_city['City_name'] = 'Other City'


In [64]:
work(tech_city, other_city)

tech_city = tech_city.drop_duplicates()
other_city = other_city.drop_duplicates()

/tmp/ipykernel_71677/2737076903.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[_bronze, ['Price']] = 'Bronze'
/tmp/ipykernel_71677/2737076903.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[_bronze, ['Price']] = 'Bronze'
/tmp/ipykernel_71677/72417587.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [65]:
_techCity = []
_otherCity = []

for i in range(len(tech_city)):
    for j in range(len(other_city)):
        if tech_city.iloc[i]['Property_type'] == other_city.iloc[j]['Property_type'] and tech_city.iloc[i]['Price'] == other_city.iloc[j]['Price'] and tech_city.iloc[i]['Size'] == other_city.iloc[j]['Size']:
            _techCity.append(i)

for i in range(len(other_city)):
    for j in range(len(tech_city)):
        if other_city.iloc[i]['Property_type'] == tech_city.iloc[j]['Property_type'] and other_city.iloc[i]['Price'] == tech_city.iloc[j]['Price'] and other_city.iloc[i]['Size'] == tech_city.iloc[j]['Size']:
            _otherCity.append(i)

tech_city = tech_city.drop(tech_city.index[_techCity])
other_city = other_city.drop(other_city.index[_otherCity])

In [66]:
tech_city

,Property_type,Property_status,City_name,Price,Size
475,Residential Plot,Under Construction,Tech City,Silver,Large


We infer the rule *"Residential Plot in Tech cities are of silver price and large size"*. Converting it into a logic formula, we get:

$$\forall \; (x) \; \text{Tech City}(x) <= \text{Property Type}(x) \in \text{Residential Plot} \wedge \text{Price}(x) \in \text{Silver} \wedge \text{Size}(x) \in \text{Large}$$